In [1]:
from neo4j import GraphDatabase
import numpy as np
import altair as alt
import pandas as pd
from tqdm import tqdm

In [2]:
import torch
import tune
import math

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from pandas.plotting import register_matplotlib_converters
import datetime
from functools import partial, reduce
import plotly.express as px
import plotly.graph_objects as go
import sklearn.model_selection as sk
import random

import pandas as pd
import numpy as np
import pandas as pd
import sklearn.model_selection as sk
import math
from typing import Tuple
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from torch import nn, Tensor
%matplotlib notebook
import matplotlib.pyplot as plt

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from sklearn.metrics import mean_squared_error
from math import sqrt

RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from tqdm import tqdm
import math

import random
import math
import time

import torch.utils.data as data_utils


In [3]:
def split_series(series, n_past, n_future):
    X, y = list(), list()
    forecast, target = list(), list()
    for window_start in range(n_past,len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end-n_past > len(series):
            break
        past = series[(window_start-n_past):(past_end-n_past), :]
        future = series[(past_end-n_past):(future_end-n_past), :]
        X.append(np.array([past]).T.tolist())
        y.append(np.array([future]).T.tolist())
    return np.array(X), np.array(y)

def shift_sequence(X_train, y_train, X_test, y_test, window):
    X_train = X_train[window:,:,:]
    X_train_list = X_train.tolist()
    for i in range (0,window):
        X_train_list.append(X_test[i,:,:].tolist())
    X_train = np.array(X_train_list)
    y_train = y_train[window:,:,:]
    y_train_list = y_train.tolist()
    for i in range(0,window):
        y_train_list.append(y_test[i,:,:].tolist())
    y_train = np.array(y_train_list)
    X_test = X_test[window:,:,:]
    y_test = y_test[window:,:,:]
    return X_train, y_train, X_test,y_test
    
def df_style(val):
    return "font-weight: bold"

def smape(y_true,y_pred):
    return round((+1/len(y_true) * np.sum(2 * np.abs(y_pred-y_true) / (np.abs(y_true) + np.abs(y_pred))*100)),1)

def mape(y_true,y_pred):
    return round((np.mean(np.abs((y_true - y_pred) / y_true)) * 100),1)

def mae(y_true,y_pred):
    error = 0
    for i in range(len(y_true)):
        error += abs(y_true[i] - y_pred[i])
    return round((float((error / len(y_true)))),1)

def rmse(y_true,y_pred):
    return round((sqrt(mean_squared_error(y_true, y_pred))), )

def rsquared(y_true,y_pred):
    RSS =  np.sum(np.square( y_true- y_pred ))
    TSS = np.sum(np.square( y_true - np.mean(y_true) ) )
    return round((1. - RSS/(TSS)), 2)

def generate_square_subsequent_mask(dim1: int, dim2: int, dim3: int) -> Tensor:
    return torch.triu(torch.ones(dim1, dim2, dim3) * float('-inf'), diagonal=1)

def log_trans(x):
    if x == 0:
        return 0
    else:
        return np.log(x)
    
embed_dim = 16

In [4]:
data = pd.read_csv('GA_with_embeddings_v2.csv')
col = len(data.columns)
data

,date,new_deaths,confirmed,people_tested,people_hospitalized,embed_0,embed_1,embed_2,embed_3,embed_4,...,embed_6,embed_7,embed_8,embed_9,embed_10,embed_11,embed_12,embed_13,embed_14,embed_15
0,2020-04-13,644.0,6716.0,16756.0,118.0,-0.005882,-0.002044,-0.003487,0.114765,-0.002321,...,-0.002520,-0.002472,0.130838,0.160536,-0.000572,-0.001984,0.193347,-0.003035,-0.004255,-0.005472
1,2020-04-14,785.0,7271.0,20786.0,3489.0,-0.005882,-0.002044,-0.003487,0.114773,-0.002321,...,-0.002520,-0.002472,0.130835,0.160534,-0.000572,-0.001984,0.193332,-0.003035,-0.004255,-0.005472
2,2020-04-15,758.0,11434.0,26869.0,0.0,-0.005882,-0.002044,-0.003487,0.114758,-0.002321,...,-0.002520,-0.002473,0.130842,0.160537,-0.000572,-0.001984,0.193365,-0.003035,-0.004255,-0.005472
3,2020-04-16,3263.0,9237.0,24567.0,4249.0,-0.005882,-0.002044,-0.003487,0.114763,-0.002321,...,-0.002520,-0.002472,0.130840,0.160536,-0.000572,-0.001984,0.193355,-0.003035,-0.004255,-0.005472
4,2020-04-17,2299.0,6906.0,0.0,1974.0,-0.005882,-0.002044,-0.003488,0.114777,-0.002321,...,-0.002520,-0.002472,0.130834,0.160534,-0.000572,-0.001984,0.193322,-0.003035,-0.004255,-0.005472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,2022-08-08,21.0,9548.0,0.0,0.0,-0.005880,-0.002045,-0.003501,0.115258,-0.002328,...,-0.002518,-0.002464,0.130666,0.160461,-0.000574,-0.001986,0.192326,-0.003046,-0.004256,-0.005466
848,2022-08-09,66.0,4982.0,0.0,0.0,-0.005880,-0.002045,-0.003501,0.115258,-0.002328,...,-0.002518,-0.002464,0.130666,0.160461,-0.000574,-0.001986,0.192326,-0.003046,-0.004256,-0.005466
849,2022-08-10,33.0,6251.0,0.0,0.0,-0.005880,-0.002045,-0.003499,0.115208,-0.002327,...,-0.002518,-0.002465,0.130684,0.160469,-0.000574,-0.001986,0.192430,-0.003045,-0.004256,-0.005467
850,2022-08-11,48.0,7576.0,0.0,0.0,-0.005880,-0.002045,-0.003499,0.115208,-0.002327,...,-0.002518,-0.002465,0.130684,0.160469,-0.000574,-0.001986,0.192430,-0.003045,-0.004256,-0.005467


In [5]:
observed = data[['date','new_deaths']]
observed

,date,new_deaths
0,2020-04-13,644.0
1,2020-04-14,785.0
2,2020-04-15,758.0
3,2020-04-16,3263.0
4,2020-04-17,2299.0
...,...,...
847,2022-08-08,21.0
848,2022-08-09,66.0
849,2022-08-10,33.0
850,2022-08-11,48.0


In [6]:
scalers={}
for i in range(0,5):
    if (i == 0):
        continue
    scaler = StandardScaler()
    scaler.fit(data.iloc[0:round(0.4*data.shape[0]),i].values.reshape(-1,1))
    s_s = scaler.transform(data.iloc[:,i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ data.columns[i]] = scaler
    data.iloc[:,i]=pd.DataFrame(s_s)

In [7]:
#without embeddings
class NN(nn.Module):
    def __init__(self,units_layer1,units_layer2,n_past,n_future,n_features,act_function1,act_function2):
        super(NN, self).__init__()
        self.act_function1 = act_function1
        self.act_function2 = act_function2
        self.linear1 = nn.Linear(n_features, embed_dim)
        self.linear2 = nn.Linear((n_past*embed_dim), units_layer1)
        self.linear3 = nn.Linear(units_layer1, units_layer2)
        self.linear4 = nn.Linear(units_layer2, n_future)        
        
    def forward(self, x):
        output = self.linear1(torch.squeeze(x[:,:,0:4,:]))             
        output = torch.flatten(torch.squeeze(output), start_dim=1)
    
        if(self.act_function1=='relu'):
            output = torch.relu(self.linear2(output))
        elif(self.act_function1=='elu'):
            output = torch.elu(self.linear2(output))
        else:
            output = torch.sigmoid(self.linear2(output))
            
        if(self.act_function2=='relu'):
            output = torch.relu(self.linear3(output))
        elif(self.act_function2=='elu'):
            output = torch.elu(self.linear3(output))
        else:
            output = torch.sigmoid(self.linear3(output))
            
        output = self.linear4(output)
        return output

In [8]:
"""
#with embeddings at beginning
class NN(nn.Module):
    def __init__(self,units_layer1,units_layer2,n_past,n_future,n_features,act_function1,act_function2):
        super(NN, self).__init__()
        self.act_function1 = act_function1
        self.act_function2 = act_function2
        self.linear1 = nn.Linear(n_features, units_layer1)
        self.linear2 = nn.Linear(units_layer1, units_layer2)
        self.linear3 = nn.Linear(units_layer2, embed_dim)
        self.linear4 = nn.Linear(112, n_future)
        
        nn.init.xavier_uniform_(self.linear1.weight)
        nn.init.xavier_uniform_(self.linear2.weight)  
        nn.init.xavier_uniform_(self.linear3.weight)
        nn.init.xavier_uniform_(self.linear4.weight) 
        
    def forward(self, x):
        output = self.linear1(torch.squeeze(x[:,:,0:4,:]))  
        
    
        if(self.act_function1=='relu'):
            output = torch.relu(self.linear2(output))

        elif(self.act_function1=='elu'):
            output = torch.elu(self.linear2(output))
        else:
            output = torch.sigmoid(self.linear2(output))
            
        if(self.act_function2=='relu'):
            output = torch.relu(self.linear3(output))

        elif(self.act_function2=='elu'):
            output = torch.elu(self.linear3(output))
        else:
            output = torch.sigmoid(self.linear3(output))
        
        
        output = output + x[:,:,4:x.shape[2]+1].squeeze()
        
        output = self.linear4(torch.flatten(output, start_dim=1))
        return output
"""

"\n#with embeddings at beginning\nclass NN(nn.Module):\n    def __init__(self,units_layer1,units_layer2,n_past,n_future,n_features,act_function1,act_function2):\n        super(NN, self).__init__()\n        self.act_function1 = act_function1\n        self.act_function2 = act_function2\n        self.linear1 = nn.Linear(n_features, units_layer1)\n        self.linear2 = nn.Linear(units_layer1, units_layer2)\n        self.linear3 = nn.Linear(units_layer2, embed_dim)\n        self.linear4 = nn.Linear(112, n_future)\n        \n        nn.init.xavier_uniform_(self.linear1.weight)\n        nn.init.xavier_uniform_(self.linear2.weight)  \n        nn.init.xavier_uniform_(self.linear3.weight)\n        nn.init.xavier_uniform_(self.linear4.weight) \n        \n    def forward(self, x):\n        output = self.linear1(torch.squeeze(x[:,:,0:4,:]))  \n        \n    \n        if(self.act_function1=='relu'):\n            output = torch.relu(self.linear2(output))\n\n        elif(self.act_function1=='elu'):\n

In [9]:
"""
#with embeddings at end
class NN(nn.Module):
    def __init__(self,units_layer1,units_layer2,n_past,n_future,n_features,act_function1,act_function2):
        super(NN, self).__init__()
        self.act_function1 = act_function1
        self.act_function2 = act_function2
        self.linear1 = nn.Linear(n_features, units_layer1)
        self.linear2 = nn.Linear(units_layer1, units_layer2)
        self.linear3 = nn.Linear(units_layer2, embed_dim)
        self.linear4 = nn.Linear(112, n_future)
        
        nn.init.xavier_uniform_(self.linear1.weight)
        nn.init.xavier_uniform_(self.linear2.weight)  
        nn.init.xavier_uniform_(self.linear3.weight)
        nn.init.xavier_uniform_(self.linear4.weight) 
        
    def forward(self, x):
        output = self.linear1(torch.squeeze(x[:,:,0:4,:]))  
        
    
        if(self.act_function1=='relu'):
            output = torch.relu(self.linear2(output))

        elif(self.act_function1=='elu'):
            output = torch.elu(self.linear2(output))
        else:
            output = torch.sigmoid(self.linear2(output))
            
        if(self.act_function2=='relu'):
            output = torch.relu(self.linear3(output))

        elif(self.act_function2=='elu'):
            output = torch.elu(self.linear3(output))
        else:
            output = torch.sigmoid(self.linear3(output))
        
        output = output + x[:,:,4:20].squeeze()
        
        output = self.linear4(torch.flatten(output, start_dim=1))
        return output
"""

"\n#with embeddings at end\nclass NN(nn.Module):\n    def __init__(self,units_layer1,units_layer2,n_past,n_future,n_features,act_function1,act_function2):\n        super(NN, self).__init__()\n        self.act_function1 = act_function1\n        self.act_function2 = act_function2\n        self.linear1 = nn.Linear(n_features, units_layer1)\n        self.linear2 = nn.Linear(units_layer1, units_layer2)\n        self.linear3 = nn.Linear(units_layer2, embed_dim)\n        self.linear4 = nn.Linear(112, n_future)\n        \n        nn.init.xavier_uniform_(self.linear1.weight)\n        nn.init.xavier_uniform_(self.linear2.weight)  \n        nn.init.xavier_uniform_(self.linear3.weight)\n        nn.init.xavier_uniform_(self.linear4.weight) \n        \n    def forward(self, x):\n        output = self.linear1(torch.squeeze(x[:,:,0:4,:]))  \n        \n    \n        if(self.act_function1=='relu'):\n            output = torch.relu(self.linear2(output))\n\n        elif(self.act_function1=='elu'):\n      

In [10]:
forecast_matrix = pd.DataFrame(columns = ['date'])
model = NN(32, 64, 7, 14, 4, 'relu','relu').to(device)
X, y = split_series(data.values, 7, 14)
X = np.swapaxes(X, 1,2)
y = np.swapaxes(y, 1,2)

t_ratio = 0.6

X_train, X_test, y_train, y_test = sk.train_test_split(X,y,test_size=t_ratio,random_state=42,shuffle=False)
y_test_temp = y_test

for i in tqdm(range(math.ceil(t_ratio*853)-7-14)):

    n_epochs_stop = 5
    epochs_no_improve = 0
    early_stop = False
    min_val_loss = np.Inf

    results = pd.DataFrame(columns = ['date'])
    X_train_dates = pd.DataFrame(X_train[0,:,0]) 
    y_train_dates = pd.DataFrame(y_train[0,:,0]) 
    X_test_dates = pd.DataFrame(X_test[0,:,0]) 
    y_test_dates = pd.DataFrame(y_test[0,:,0])

    X_train_new = torch.from_numpy(np.array(X_train[:,:,1:col+embed_dim],dtype=np.float32)).float()
    y_train_new = torch.from_numpy(np.array(y_train[:,:,1:col+embed_dim],dtype=np.float32)).float()
    X_test_new = torch.from_numpy(np.array(X_test[:,:,1:col+embed_dim],dtype=np.float32)).float()
    y_test_new = torch.from_numpy(np.array(y_test[:,:,1:col+embed_dim],dtype=np.float32)).float()

    X_train_new = X_train_new.to(device)
    y_train_new = y_train_new.to(device)
    X_test_new = X_test_new.to(device)
    y_test_new = y_test_new.to(device)

    results['date'] = y_test_dates 

    loss_fn = torch.nn.L1Loss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    train = data_utils.TensorDataset(X_train_new,y_train_new)
    train_loader = data_utils.DataLoader(train, batch_size=32, shuffle=True)

    for epoch in range(100):
        train_losses = []
        model.train()

        for x_t, y_t in train_loader:
            optimizer.zero_grad()

            y_pred = model(x_t)
            loss = loss_fn(y_pred.float(), torch.squeeze(y_t[:,:,0]))

            loss.backward()

            optimizer.step()

            train_losses.append(loss.item())

        train_loss = round(np.average(train_losses),2)

        if train_loss < min_val_loss:
            epochs_no_improve = 0
            min_val_loss = train_loss
        else:
            epochs_no_improve += 1

        if epochs_no_improve == n_epochs_stop:
            break

    with torch.no_grad():
        model.eval()
        y_test_pred = model(X_test_new)
        results['window'+str(i)] = y_test_pred[0,:].cpu().flatten().numpy()
    X_train, y_train, X_test, y_test = shift_sequence(X_train,y_train,X_test,y_test,1)
    if(i==0):
        forecast_matrix = pd.merge(forecast_matrix, results, on=['date'], how='right')
    else:
        forecast_matrix = pd.merge(forecast_matrix, results, on=['date'], how='outer')

forecast_matrix.head()
date = pd.DataFrame(forecast_matrix['date'])
forecast_matrix.set_index('date', inplace=True)

scaler = scalers['scaler_new_deaths']
forecast_matrix=scaler.inverse_transform(forecast_matrix)
forecast_matrix =pd.DataFrame(forecast_matrix)
forecast_matrix['date'] = date
forecast_matrix.set_index('date', inplace=True)
forecast_matrix.shape[0]
start = 0
end = 14
daily_predictions = []
window = 0
for i in range(len(forecast_matrix.columns)-13):
    forecast_matrix_temp = forecast_matrix.iloc[start:end]
    date = forecast_matrix_temp.tail(1).index.item()
    l_row = forecast_matrix_temp.tail(1)
    daily_predictions.append([date,l_row[window][0],
                             l_row[window+1][0],l_row[window+2][0],l_row[window+3][0],
                             l_row[window+4][0],l_row[window+5][0],l_row[window+6][0],
                             l_row[window+7][0],l_row[window+8][0],l_row[window+9][0],
                             l_row[window+10][0],l_row[window+11][0],l_row[window+12][0],
                             l_row[window+13][0]])

    window = window + 1
    start = end 
    end = end+1    
daily_predictions = pd.DataFrame(daily_predictions, columns=[ 'date','day14_NN','day13_NN','day12_NN',
                                                              'day11_NN','day10_NN','day9_NN','day8_NN',
                                                              'day7_NN','day6_NN','day5_NN','day4_NN',
                                                              'day3_NN','day2_NN','day1_NN'])
daily_predictions['date'] = pd.to_datetime(daily_predictions['date'])
daily_nn = daily_predictions

100%|████████████████████████████████████████████████████████████████████████████████| 491/491 [03:24<00:00,  2.40it/s]


In [11]:
observed.rename(columns={'new_deaths': 'observed'}, inplace=True)
observed.reset_index(inplace=True, drop= True)
observed['date'] = pd.to_datetime(observed['date'])
matrix = reduce(lambda x,y: pd.merge(x,y, on='date'), [observed, daily_nn])
matrix = matrix[matrix['observed'] != 0]
matrix.reset_index(inplace=True, drop= True)

C:\Users\maldo\AppData\Local\Temp\ipykernel_5496\3433566817.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  observed.rename(columns={'new_deaths': 'observed'}, inplace=True)
C:\Users\maldo\AppData\Local\Temp\ipykernel_5496\3433566817.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  observed['date'] = pd.to_datetime(observed['date'])


In [12]:
#---------MAE------------#
results_mae = {}
results_mae
for i in ('NN','NN'):
    results_mae[i] = []
    for j in('day1','day2','day3','day4','day5','day6','day7','day8','day9','day10','day11','day12',
            'day13','day14'):
        results_mae[i].append(mae(matrix['observed'],matrix[j+'_'+i]))
results_mae = pd.DataFrame(results_mae)
results_mae.index += 1 
results_mae.loc['Average'] = results_mae.mean()
last_row = pd.IndexSlice[results_mae.index[results_mae.index == "Average"], :]
results_mae.style.applymap(df_style, subset=last_row)

,NN
1,17.800000
2,17.100000
3,18.600000
4,18.800000
5,18.000000
6,17.500000
7,16.700000
8,17.700000
9,18.000000
10,18.300000


In [13]:
#---------SMAPE------------#
results_smape = {}
results_smape
for i in ('NN','NN'):
    results_smape[i] = []
    for j in('day1','day2','day3','day4','day5','day6','day7','day8','day9','day10','day11','day12',
            'day13','day14'):
        results_smape[i].append(smape(matrix['observed'],matrix[j+'_'+i]))
results_smape = pd.DataFrame(results_smape)
results_smape.index += 1 
results_smape.loc['Average'] = results_smape.mean()
last_row = pd.IndexSlice[results_smape.index[results_smape.index == "Average"], :]
results_smape.style.applymap(df_style, subset=last_row)

,NN
1,45.800000
2,44.100000
3,44.700000
4,45.900000
5,43.300000
6,42.800000
7,42.900000
8,44.300000
9,45.400000
10,44.800000
